<a href="https://colab.research.google.com/github/omarespejel/Hugging-Face-101-ES/blob/main/1_NLP_en_espa%C3%B1ol%3A_Importando_un_modelo_y_tokenizando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. NLP en español: Importando un modelo y tokenizando
por Omar U. Espejel (Twitter: [@espejelomar](https://twitter.com/espejelomar))


- Puedes escribirme vía Twitter en [@espejelomar](https://twitter.com/espejelomar?lang=en) 🐣. 

- Únete al [Discord de Hugging Face](https://t.co/1n75wi976V?amp=1).
  
- Checa el [diccionario inglés-español](https://www.notion.so/Ingl-s-para-la-programaci-n-bab11d9db5014f16b840bf8d22c23ac2) para programación.

El material aquí presente está inspirado por el modelo BETO originalmente en el [repositorio del Departamento de Ciencias de la Computación de la Universidad de Chile](https://github.com/dccuchile).


## Instalación de BETO

Primero instalamos BETO desde HuggingFace Hub. El repositorio con mayor cantidad de modelos open source.

In [3]:
%%capture
!pip install transformers

In [4]:
import torch
from transformers import BertForMaskedLM, BertTokenizer

Observamos que en efecto los datos correspondientes a BETO se encuentran en la carpeta pytorch.

In [5]:
!ls pytorch/

ls: cannot access 'pytorch/': No such file or directory


Si fueramos a utilizar el modelo BERT original, ya en la instalación de `transformers` que hicimos arriba, usaríamos el siguiente comando:

In [6]:
# tokenizer_ingles = BertTokenizer.from_pretrained('bert-base-cased')

Para utilizar BETO tenemos que importar los datos que deliberadamente guardamos en la carpeta pyorch de nuestro ambiente. El tokenizador lo dejamos con `do_lower_case` para que sí podamos aceptar palabras con mayúsculas.

In [7]:
tokenizer_español = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased", do_lower_case=False)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/170k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [8]:
tokenizer_español

PreTrainedTokenizer(name_or_path='Geotrend/distilbert-base-es-cased', vocab_size=26359, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [9]:
BertTokenizer??

Si observamos el vocabulario con el que BETO está pre-entrenado observamos:



*  Primeras 977 palabras están reservadas en la forma [unusedK] y [MASK], [PAD], [EOS], [UNK], [CLS], [SEP]. Las celdas unused son para que agreguemos vocabulario que puede ser específico para nuestra aplicación.
* A partir del 978 vemos tokens para caracteres individuales, como números y letras.
*   Poco a poco comienzan surgir tokens que individualmente pueden ser palabras. Están ordenadas por frecuencia.
*   Hay palabras que pensaríamos no tan comunes en este contexto como "verga" e "hincha". ¿Cómo expresas la palabra verga en inglés? Por eso necesitamos nuestro propio vocabulario en español.
*   Todo el tiempo también vemos las subpalabras que comienzan con #








## *Tokenizing* con BETO

Tokenizer en acción

In [10]:
enunciado = "BETO es clave para el desarrollo del NLP en América Latina."

In [11]:
print('Original: ', enunciado)
print("Tokenizado: ", tokenizer_español.tokenize(enunciado))
print('IDs: ', tokenizer_español.convert_tokens_to_ids(tokenizer_español.tokenize(enunciado)))

Original:  BETO es clave para el desarrollo del NLP en América Latina.
Tokenizado:  ['BE', '##TO', 'es', 'clave', 'para', 'el', 'desarrollo', 'del', 'NL', '##P', 'en', 'América', 'Latina', '.']
IDs:  [13047, 16755, 294, 15271, 315, 225, 5263, 227, 21292, 887, 211, 2733, 8293, 27]


Es más rápido si usamos `tokenizer_español.encode()` para convertir el texto primero en tokens y luego en IDs.

También se puede puede incluir el texto directamente en `tokenizer_español()` y nos retornará un diccionario en donde la *key* `input_ids` incluye los IDs que `tokenizer_español.encode()` también nos retornaría.







In [12]:
tokenizer_español(enunciado)['input_ids']

[11,
 13047,
 16755,
 294,
 15271,
 315,
 225,
 5263,
 227,
 21292,
 887,
 211,
 2733,
 8293,
 27,
 12]

In [26]:
print(f'Este es el resultado de usar tokenizer_español.encode: {tokenizer_español.encode(enunciado)}\n')
print(f'Este es el resultado de usar solo tokenizer_español: {tokenizer_español(enunciado)["input_ids"]}\n')
print('Es exactamente lo mismo!!')

Este es el resultado de usar tokenizer_español.encode: [11, 13047, 16755, 294, 15271, 315, 225, 5263, 227, 21292, 887, 211, 2733, 8293, 27, 12]

Este es el resultado de usar solo tokenizer_español: [11, 13047, 16755, 294, 15271, 315, 225, 5263, 227, 21292, 887, 211, 2733, 8293, 27, 12]

Es exactamente lo mismo!!


Si tenemos dos textos se tendrá que crear un padding para convertir al mismo tamaño los tensores,

In [27]:
texto_corto = "Este texto es corto"
texto_largo = "Este texto es largo y un poco aburrido"

corto_encoded = tokenizer_español(texto_corto)["input_ids"]
largo_encoded = tokenizer_español(texto_largo)["input_ids"]

len(corto_encoded), len(largo_encoded)

(6, 11)

In [28]:
secuencia_con_padding = tokenizer_español([texto_corto, texto_largo], padding = True)

El padding lo notamos en en la `key` de `secuencia_con_padding` llamada `attention_mask`. Profundizemos en esto.

In [16]:
secuencia_con_padding

{'input_ids': [[11, 1648, 7552, 294, 13429, 12, 0, 0, 0, 0, 0], [11, 1648, 7552, 294, 3646, 101, 220, 2578, 25465, 18834, 12]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

Como tal, tokenizer nos devuelve un diccionario con tres keys: `'input_ids', 'token_type_ids', 'attention_mask'`. En este momento solo queremos mostrar `input_ids`. Interesantemente, con BETO el padding se realiza con el token especial `[PAD]` (el 1) en vez de con un cero, pues esta posición está reservada para el token especial `[MASK]`.

In [29]:
secuencia_con_padding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [18]:
secuencia_con_padding['input_ids']

[[11, 1648, 7552, 294, 13429, 12, 0, 0, 0, 0, 0],
 [11, 1648, 7552, 294, 3646, 101, 220, 2578, 25465, 18834, 12]]

Podemos usar un el método `decode` de nuestro tokenizer para observar lo que cada id significa.

In [19]:
tokenizer_español.decode(secuencia_con_padding['input_ids'][0]), tokenizer_español.decode(secuencia_con_padding['input_ids'][1])

('[CLS] Este texto es corto [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] Este texto es largo y un poco aburrido [SEP]')

Con `attention_mask` podemos ver la parte de los enunciados que tuvieron padding. A pesar de que el padding se realizó con el token 1 en vez del 0, notamos que la `attention_mask` detecta sin nigún problema donde se realizó el padding.

In [20]:
secuencia_con_padding['attention_mask']

[[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

La última key producto de aplicar tokenizer es `token_type_ids`. Esta nos ayudará para tareas como clasificación de secuencias o para responder preguntas. Lo que hacemos es unir nuestros textos en una sola secuencia con ayuda de los tokens especiales 5 `[CLS]` y 6 `[SEP]`. 

El modelo junta ambas secuencias en un único tensor. Por ejemplo, en el caso de resolver una pregunta, la pregunta quedaría con 0s y la respuesta con 1s en nuestra `token_type_ids`.

In [21]:
secuencia = tokenizer_español("Esta clase es sobre cómo utilizar BETO", "¿Sobre qué es esta clase?")

In [22]:
secuencia["token_type_ids"], tokenizer_español.decode(secuencia['input_ids'])

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 '[CLS] Esta clase es sobre cómo utilizar BETO [SEP] ¿ Sobre qué es esta clase? [SEP]')

Notamos que nos une ambos enunciados en uno solo, no hay necesidad de usar padding.

## Lo que sigue...

En el siguiente *notebook* importaremos un dataset original y lo preparemos con BETO para clasificación.